<a href="https://colab.research.google.com/github/FatemeZamanian/DeepLearning/blob/main/Flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

In [3]:
dataset_path="/content/drive/MyDrive/Flowers"
width=height=224
batch_size=32
data_generator=ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=10,
    zoom_range=0.1,
    validation_split=0.1
)

In [46]:
train_data=data_generator.flow_from_directory(
    os.path.join(dataset_path,'Train'),
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='training',

)

val_data=data_generator.flow_from_directory(
    os.path.join(dataset_path,'Train'),
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='validation',

)

test_data=data_generator.flow_from_directory(
    os.path.join(dataset_path,'Test'),
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False,

)

Found 986 images belonging to 17 classes.
Found 102 images belonging to 17 classes.
Found 272 images belonging to 17 classes.


In [47]:
# model_v=tf.keras.applications.VGG16(
#     input_shape=(width,height,3),
#     include_top=False,
#     weights='imagenet'
# )

model_r=tf.keras.applications.ResNet50V2(
    input_shape=(width,height,3),
    include_top=False,
    weights='imagenet',
    pooling='max'
)

In [48]:
# for layer in model_v.layers:
#   layer.trainable=False

for layer in model_r.layers[:-2]:
  layer.trainable=False

In [49]:
# model=tf.keras.Sequential([
#         model_v,
#         layers.Flatten(),
#         layers.Dense(1024,activation='relu'),
#         layers.Dense(128,activation='relu'),
#         layers.Dense(17,activation='softmax'),
# ])

model=tf.keras.Sequential([
        model_r,
        layers.Dense(2048,activation='relu'),
        layers.Dense(1024,activation='relu'),
        layers.Dense(128,activation='relu'),
        layers.Dense(17,activation='softmax'),
])

In [ ]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback

In [ ]:
wandb.init(project="Flowers")
config = wandb.config
config.learning_rate = 0.001

In [51]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= config.learning_rate ),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'],)

In [52]:
model.fit(train_data,
          steps_per_epoch=train_data.samples//batch_size,
          validation_data=val_data,
          validation_steps=val_data.samples//batch_size,
          epochs=15,
          # callbacks=[WandbCallback()]

    
)

Epoch 1/15
30/30 [==============================] - 25s 721ms/step - loss: 7.3697 - accuracy: 0.3229 - val_loss: 1.0614 - val_accuracy: 0.6250
Epoch 2/15
30/30 [==============================] - 20s 669ms/step - loss: 0.6551 - accuracy: 0.8008 - val_loss: 0.3838 - val_accuracy: 0.8750
Epoch 3/15
30/30 [==============================] - 20s 665ms/step - loss: 0.3029 - accuracy: 0.9088 - val_loss: 0.2428 - val_accuracy: 0.8854
Epoch 4/15
30/30 [==============================] - 20s 670ms/step - loss: 0.1675 - accuracy: 0.9403 - val_loss: 0.2881 - val_accuracy: 0.9062
Epoch 5/15
30/30 [==============================] - 20s 671ms/step - loss: 0.0904 - accuracy: 0.9717 - val_loss: 0.2329 - val_accuracy: 0.9271
Epoch 6/15
30/30 [==============================] - 20s 668ms/step - loss: 0.0899 - accuracy: 0.9717 - val_loss: 0.5635 - val_accuracy: 0.8229
Epoch 7/15
30/30 [==============================] - 20s 665ms/step - loss: 0.1657 - accuracy: 0.9350 - val_loss: 0.3521 - val_accuracy: 0.8854

In [53]:
model.evaluate(test_data)

9/9 [==============================] - 5s 576ms/step - loss: 0.7889 - accuracy: 0.8272


[0.7888979911804199, 0.8272058963775635]